In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.wrappers import Bidirectional
from embeddings import Embeddings
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
embedding = Embeddings(100, 3, 1, 4)

In [3]:
word2vec_model = embedding.get_model()
vocab = embedding.get_vocabulary()

In [4]:
word2index = vocab[0]

In [5]:
contexts = ["""Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City.
He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range of electrical devices.
His patented AC induction motor and transformer were licensed by George Westinghouse, who also hired Tesla for a short time as a consultant.
His work in the formative years of electric power development was involved in a corporate alternating current/direct current "War of Currents" as well as various patent battles.
""","""Tesla went on to pursue his ideas of wireless lighting and electricity distribution in his high-voltage, high-frequency power experiments in New York and Colorado Springs, and made early (1893) pronouncements on the possibility of wireless communication with his devices. 
          He tried to put these ideas to practical use in an ill-fated attempt at intercontinental wireless transmission, his unfinished Wardenclyffe Tower project.
          In his lab he also conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.
          He also built a wireless controlled boat, one of the first ever exhibited."""]

In [9]:
max_len = max([len(word_tokenize(context)) for context in contexts])
"""tokenized_contexts = [word_tokenize(context) for context in contexts]
indexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]"""

'tokenized_contexts = [word_tokenize(context) for context in contexts]\nindexed_contexts = [[word2index[word] for word in context if word in word2index] for context in tokenized_contexts]'

In [10]:
# padded_contexts = sequence.pad_sequences(indexed_contexts)

In [11]:
target = np.zeros(max_len*(max_len-1))

In [12]:
question = "In what year did Nikola Tesla emigrate to the United States?"

In [13]:
from itertools import product

In [14]:
context_question = list(product(contexts, [question]))

In [15]:
# tokenized_sent = [word_tokenize(sent) for sent in sent_tokenize(context)]

In [19]:
def get_average_sentence_vector(sentence):
    avg_sent_vec = np.zeros((100))
    count = 0
    for word in sentence:
        if word in word2vec_model.wv.vocab:
            avg_sent_vec += word2vec_model[word]
            count+=1
    avg_sent_vec = np.divide(avg_sent_vec, count)
    return avg_sent_vec

In [20]:
avg_sentences = []
for context, question in context_question:
    avg_sentences.append(get_average_sentence_vector(word_tokenize(context)))

In [21]:
avg_sentences = np.array(avg_sentences)

In [23]:
avg_question_vec = get_average_sentence_vector(word_tokenize(question))

In [24]:
x = np.array(list(product(avg_sentences, [avg_question_vec])))

In [25]:
x = x.reshape((2, 200))

In [26]:
sent_1_answer = target
sent_1_answer[13] = 1
sent_2_answer = target
y = np.array([sent_1_answer, sent_2_answer])

In [27]:
y.shape

(2, 11130)

In [28]:
model = Sequential()
model.add(Dense(100, input_shape=(200,)))
model.add(Dense(y.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x, y, epochs=100)

Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[100]
	 [[Node: Variable_4/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_4"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Variable_4, Const_10)]]

Caused by op 'Variable_4/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nishant/venvs/autofill/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/nishant/venvs/autofill/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/nishant/venvs/autofill/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/nishant/venvs/autofill/lib/python3.5/site-packages/zmq/eventloop/iol

In [ ]:
model.predict(np.array([x[2]]))